In [1]:
import pandas as pd
import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pickle
from datetime import datetime
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import f1_score


In [3]:
# Load data
data = pd.read_csv(FILE_PATH)

In [4]:
# Data preparation
data = data[['Close', 'SMA200', 'SMA50', 'RSI14', 'signal']]
data = data.dropna()  # Drop missing values

In [5]:
scaler = MinMaxScaler()
data[['Close', 'SMA200', 'SMA50', 'RSI14']] = scaler.fit_transform(data[['Close', 'SMA200', 'SMA50', 'RSI14']])
data = data.round(4)

In [6]:
data['signal'] = data['signal'] - 1  # Convert labels: 1 -> 0 (buy), 2 -> 1 (sell), 3 -> 2 (nothing)

In [7]:
# Dataset and DataLoader
class ForexDataset(Dataset):
    def __init__(self, data, seq_length):
        self.data = data
        self.seq_length = seq_length

    def __len__(self):
        return len(self.data) - self.seq_length

    def __getitem__(self, idx):
        x = self.data.iloc[idx:idx + self.seq_length, :-1].values
        y = self.data.iloc[idx + self.seq_length, -1]
        return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.long)

In [8]:
dataset = ForexDataset(data, seq_length)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [9]:
# Model definition
class ForexModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes, dropout):
        super(ForexModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(dropout)
        self.gelu = nn.GELU()
        self.softmax = nn.Softmax(dim=1)
        self.middle_layer = nn.Linear(hidden_size, middle_size)
        self.fc = nn.Linear(middle_size, num_classes)

    def forward(self, x):
        _, (hn, _) = self.lstm(x)
        hn = self.dropout(hn[-1])
        hn = self.middle_layer(hn)
        hn = self.gelu(hn)
        hn = self.fc(hn)
        hn = self.softmax(hn)
        return hn

In [10]:
# Evaluate the model and collect predictions and true labels
def evaluate_model(model, dataloader, device):
    model.eval()
    y_true = []
    y_pred = []
    with torch.no_grad():
        for x_batch, y_batch in dataloader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            outputs = model(x_batch)
            _, predicted = torch.max(outputs, 1)
            y_true.extend(y_batch.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())
    return y_true, y_pred

In [11]:
model = ForexModel(input_size, hidden_size, num_classes, dropout).to(device)

In [12]:
# MODEL_PATH = "D:/Programing/AI Trader/Model/lstmModelv.1.0/Model v.1.0_loss 62.9028_Acc 0.8692_at 20241224-051604.model"

# model.load_state_dict(torch.load(MODEL_PATH, weights_only=False))

# # model.eval()

# print(f"{MODEL_PATH.split('/')[-1]} is loaded.")

In [13]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [14]:
# Training loop
for epoch in tqdm(range(epochs), desc="Training Epochs"):
    total_loss = 0
    correct = 0
    total = 0
    model.train()
    for x_batch, y_batch in dataloader:
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)

        outputs = model(x_batch)
        loss = criterion(outputs, y_batch)
        total_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        _, predicted = torch.max(outputs, 1)
        correct += (predicted == y_batch).sum().item()
        total += y_batch.size(0)

    accuracy = correct / total
    
    timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
    MODEL_SAVEPATH = f"D:/Programing/AI Trader/Model/lstmModelv.1.2/Model v.1.2_loss {total_loss:.4f}_Acc {accuracy:.4f}_at {timestamp}.model"
    torch.save(model.state_dict(), MODEL_SAVEPATH)
    
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f"Epoch {epoch + 1}/{epochs}; Loss: {total_loss:.4f} , Accuracy: {accuracy:.4f} | @ {timestamp}")

Training Epochs:   1%|          | 1/100 [20:12<33:21:14, 1212.87s/it]

Epoch 1/100; Loss: 203.7256 , Accuracy: 0.5005 | @ 2024-12-27 00:09:58


Training Epochs:   2%|▏         | 2/100 [37:45<30:27:10, 1118.68s/it]

Epoch 2/100; Loss: 199.6694 , Accuracy: 0.5082 | @ 2024-12-27 00:27:31


Training Epochs:   3%|▎         | 3/100 [53:54<28:17:41, 1050.11s/it]

...

Epoch 59/100; Loss: 196.0241 , Accuracy: 0.5252 | @ 2024-12-27 15:26:06


Training Epochs:  60%|██████    | 60/100 [15:49:58<9:02:52, 814.31s/it]

Epoch 60/100; Loss: 195.9918 , Accuracy: 0.5244 | @ 2024-12-27 15:39:44


Training Epochs:  61%|██████    | 61/100 [16:07:37<9:36:50, 887.46s/it]

Epoch 61/100; Loss: 195.8771 , Accuracy: 0.5247 | @ 2024-12-27 15:57:22


In [15]:
device

device(type='cuda')

In [ ]:
timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")

MODEL_SAVEPATH = f"D:/Programing/AI Trader/Model/lstmModelv.1.0/Model v.1.0_loss {total_loss:.4f}_Acc {accuracy:.4f}_at {timestamp}.model"

print(f"{MODEL_SAVEPATH.split('/')[-1]} was saved.")

torch.save(model.state_dict(), MODEL_SAVEPATH)

In [26]:
# Evaluate on the same dataset
y_true, y_pred = evaluate_model(model, dataloader, device)

In [ ]:
# Compute confusion matrix
cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Buy (0)', 'Sell (1)', 'Nothing (2)'])

# Plot confusion matrix
plt.figure(figsize=(8, 6))
disp.plot(cmap='Blues', values_format='d')
plt.title("Confusion Matrix")
plt.show()

# image_path = "./Result/cm-01.png"
# plt.savefig(image_path)

In [ ]:
# Calculate F1 score
f1 = f1_score(y_true, y_pred, average='weighted')
print(f"F1 Score: {f1:.4f}")

In [ ]:
# Calculate F1 score
f1 = f1_score(y_true, y_pred, average='weighted')
print(f"F1 Score: {f1:.4f}")

In [ ]:
# Calculate F1 score
f1 = f1_score(y_true, y_pred, average='weighted')
print(f"F1 Score: {f1:.4f}")

In [ ]:
# Calculate F1 score
f1 = f1_score(y_true, y_pred, average='weighted')
print(f"F1 Score: {f1:.4f}")